In [4]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

In [5]:
greet_bob(say_hello)

'Hello Bob'

In [6]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

### Inner Functions

In [9]:
def parent():
    print("Printing from the parent() function")
    def first_child():
        print("Printing from the first_child() function")
    def second_child():
        print("Printing from the second_child() function")
    second_child()
    first_child()

In [10]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


### Returning Functions From Functions#


In [12]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

Below are functions

In [16]:
first = parent(1)
second = parent(2)
first
second

<function __main__.parent.<locals>.second_child()>

In [17]:
first()
second()

'Call me Liam'

#### Decorators

Wrap the function and return with same name.


Put simply: decorators wrap a function, modifying its behavior.

In [21]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

In [22]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

In [23]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

In [25]:
#If you try to call say_whee() after bedtime, nothing will happen:

say_whee()

#### Syntactic Sugar!

Python allows you to use decorators in a simpler way with the @ symbol


In [27]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

In [28]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


## Decorating Functions With Arguments


In [29]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice



@do_twice
def greet(name):
    print(f"Hello {name}")

In [30]:
greet("JV")

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The solution is to use *args and **kwargs in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments.

In [32]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice


@do_twice
def greet(name):
    print(f"Hello {name}")
    

In [33]:
greet("JV")

Hello JV
Hello JV


#### Returning Values From Decorated Functions


In [34]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [36]:
hi_adam = return_greeting("Adam")

Creating greeting
Creating greeting


Oops, your decorator ate the return value from the function.

Because the do_twice_wrapper() doesn’t explicitly return a value, the call return_greeting("Adam") ended up returning None.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your decorators.py file:

In [38]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs) ##Note here
    return wrapper_do_twice

In [39]:
return_greeting("Adam")

Creating greeting
Creating greeting


In [40]:
## a template code: Note functools used to repserve doc and function informations

import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

### Timing Functions Example


In [41]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [42]:
waste_some_time(1)


Finished 'waste_some_time' in 0.0025 secs


In [43]:
waste_some_time(999)


Finished 'waste_some_time' in 2.5306 secs


#### Debugging Code

The following @debug decorator will print the arguments a function is called with as well as its return value every time the function is called:

In [44]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

In [45]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [46]:
make_greeting("Benjamin")

Calling make_greeting('Benjamin')
'make_greeting' returned 'Howdy Benjamin!'


'Howdy Benjamin!'

In [47]:
make_greeting("Richard", age=112)

Calling make_greeting('Richard', age=112)
'make_greeting' returned 'Whoa Richard! 112 already, you are growing up!'


'Whoa Richard! 112 already, you are growing up!'

To calculate operator e

In [48]:
import math

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

In [49]:
approximate_e(5)


Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24


2.708333333333333

#### Registering Plugins


Decorators don’t have to wrap the function they’re decorating. They can also simply register that a function exists and return it unwrapped. This can be used, for instance, to create a light-weight plug-in architecture:

In [50]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

In [51]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

In [57]:
randomly_greet("Alice")

Using 'be_awesome'


'Yo Alice, together we are the awesomest!'